# Python & Fortran

You can define Fortran functions and subrouttines in a Jupyter notebook, and call those to process Python data (basic types and numpy arrays).

First, the notebook extension should be installed if that was not already done using pip:
```
!pip install fortran-magic
```
Alternatively, it can be installed using conda from the conda-forge channel:
```
!conda install -c conda-forge fortran-magic
```
Once this is done, the extension can be loaded.

In [1]:
%load_ext fortranmagic

/home/gjb/miniconda3/envs/py36/lib/python3.6/site-packages/fortranmagic.py:147: UserWarning: get_ipython_cache_dir has moved to the IPython.paths module since IPython 4.0.
  self._lib_dir = os.path.join(get_ipython_cache_dir(), 'fortran')


We can define a Fortran subroutine.  There are some restrictions on the subroutine's signature since it will behave as a function when called in Python context. The single `intent(out)` argument should not be passed in, and its value will be returned as if it were a function.

In [4]:
%%fortran
subroutine bla(x, r)
    implicit none
    integer, intent(in) :: x
    integer, intent(out) :: r
    r = x + 1
end subroutine

Once the subroutine is defined, it can be used as any Python function.

In [6]:
a = 15
b = 0
b = bla(a)
print(a, b)

15 16


## OpenMP

Check the number of threads for OpenMP, if it is 1, things will not be very interesting.  Set before starting the notebook using:
```
export OMP_NUM_THREADS=3
```

In [2]:
import os
os.environ['OMP_NUM_THREADS']

'3'

A very simple subroutine that illustrates that OpenMP works.  It returns the number of threads, and the product $\prod_{t_{\rm id}=0}^{t_{\rm num} - 1} (t_{\rm id} + 1)$ for $t_{\rm id}$ the thread ID and $t_{\rm num}$ the number of threads.

In [18]:
%%fortran --f90flags '-fopenmp'
subroutine test_omp(num_threads, thread_num_prod)
    use omp_lib
    implicit none
    integer, intent(out) :: num_threads, thread_num_prod
    integer :: thread_num
    thread_num_prod = 1
    !$omp parallel private(thread_num), shared(num_threads), reduction(*:thread_num_prod)
        thread_num = omp_get_thread_num()
        if (thread_num == 0) &
            num_threads = omp_get_num_threads()
        thread_num_prod = thread_num_prod*(1 + thread_num)
    !$omp end parallel
end subroutine test_omp

For three threads, the results should be 3 and 6 respectively.

In [19]:
test_omp()

(3, 6)